In [36]:
import pandas as pd
import datetime as dt

In [37]:
#Load datasets
url = "./Downloads/train(1).csv" 
df = pd.read_csv(url)

file = "./Downloads/metadata.csv"
dfMeta = pd.read_csv(file, sep=";")

In [38]:
#Cast to timestamp

df['tunein']= pd.to_datetime(df['tunein'], format='%Y-%m-%d %H:%M:%S')
df['tuneout']= pd.to_datetime(df['tuneout'], format='%Y-%m-%d %H:%M:%S')

In [39]:
df['minVisualizados'] = df['tuneout']-df['tunein']

In [40]:
df['minVisualizados']= df['minVisualizados'].dt.total_seconds().div(60).astype(int)

In [41]:
df

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume,minVisualizados
0,0,90627,STATIONARY,18332.0,2021-02-18 22:52:00,2021-02-18 23:35:00,0,43
1,0,90627,STATIONARY,24727.0,2021-03-24 23:17:00,2021-03-25 00:01:00,0,44
2,1,3387,STB,895.0,2021-03-15 10:05:00,2021-03-15 10:23:00,0,18
3,1,3387,STB,895.0,2021-03-15 10:23:00,2021-03-15 11:18:00,1,55
4,1,3387,STB,26062.0,2021-03-16 09:24:00,2021-03-16 09:44:00,0,20
...,...,...,...,...,...,...,...,...
3657796,112339,3386,STB,330.0,2021-03-31 22:10:00,2021-03-31 22:22:00,0,12
3657797,112339,3386,STB,9931.0,2021-03-31 22:22:00,2021-03-31 22:34:00,1,12
3657798,112339,3386,STB,29929.0,2021-03-31 22:34:00,2021-03-31 22:46:00,0,12
3657799,112339,3386,STB,29929.0,2021-03-31 23:09:00,2021-03-31 23:13:00,1,4


In [44]:
df.asset_id.isna().sum() 

0

In [43]:
#Remove null values
df=df.dropna(subset=['asset_id'])

In [45]:
#Cast asset_id to int to be able to merge
df = df.astype({'asset_id':'int'})

In [46]:
dfMetaMinutos = dfMeta[['asset_id', 'run_time_min']]

In [47]:
#Cast run_time_min to compare same datatype columns
dfMetaMinutos = dfMetaMinutos.astype({'run_time_min':'int'})

In [48]:
#Merge the two DF in one 
result =pd.merge(df,dfMetaMinutos, how='inner',on ='asset_id') #ver tipo de merge

In [49]:
result

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume,minVisualizados,run_time_min
0,0,90627,STATIONARY,18332,2021-02-18 22:52:00,2021-02-18 23:35:00,0,43,43
1,3,3389,CLOUD_CLIENT,18332,2021-02-27 16:25:00,2021-02-27 17:05:00,0,40,43
2,11,3395,STATIONARY,18332,2021-02-24 17:24:00,2021-02-24 18:10:00,0,46,43
3,13,26042,PHONE,18332,2021-02-24 21:56:00,2021-02-24 22:44:00,0,48,43
4,35,26060,STB,18332,2021-02-19 15:10:00,2021-02-19 15:15:00,1,5,43
...,...,...,...,...,...,...,...,...,...
3657774,111822,25640,STB,15863,2021-03-16 00:25:00,2021-03-16 00:28:00,1,3,110
3657775,111822,25640,STB,15598,2021-03-26 01:11:00,2021-03-26 02:22:00,1,71,107
3657776,111822,25640,STB,22513,2021-03-27 03:56:00,2021-03-27 05:05:00,1,69,107
3657777,111988,25777,PHONE,24720,2021-02-05 11:54:00,2021-02-05 11:57:00,0,3,89


In [50]:
result['target'] = ((result.minVisualizados > result.run_time_min*0.5) | (result.resume == 1))

In [51]:
result

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume,minVisualizados,run_time_min,target
0,0,90627,STATIONARY,18332,2021-02-18 22:52:00,2021-02-18 23:35:00,0,43,43,True
1,3,3389,CLOUD_CLIENT,18332,2021-02-27 16:25:00,2021-02-27 17:05:00,0,40,43,True
2,11,3395,STATIONARY,18332,2021-02-24 17:24:00,2021-02-24 18:10:00,0,46,43,True
3,13,26042,PHONE,18332,2021-02-24 21:56:00,2021-02-24 22:44:00,0,48,43,True
4,35,26060,STB,18332,2021-02-19 15:10:00,2021-02-19 15:15:00,1,5,43,True
...,...,...,...,...,...,...,...,...,...,...
3657774,111822,25640,STB,15863,2021-03-16 00:25:00,2021-03-16 00:28:00,1,3,110,True
3657775,111822,25640,STB,15598,2021-03-26 01:11:00,2021-03-26 02:22:00,1,71,107,True
3657776,111822,25640,STB,22513,2021-03-27 03:56:00,2021-03-27 05:05:00,1,69,107,True
3657777,111988,25777,PHONE,24720,2021-02-05 11:54:00,2021-02-05 11:57:00,0,3,89,False


In [52]:
result.target.value_counts()

True     3013348
False     644431
Name: target, dtype: int64